#  基于MindSpore实现LSTM算法 
文本生成是自然语言处理领域的一个重要研究方向，研究人员期待有一天计算机能够像人类一样会表达，能够撰写出高质量的自然语言文本。
## 1 实验目的
本章的主要内容就是基于 MindSpore 实现文本生成，通过本实验，学员可以学习LSTM网络的使用，文本的预处理操作。熟悉循环神经网络的结构，文本生成的过程以及网络模型的训练和预测，掌握 MindSpore 的相关操作等
## 2 LSTM算法原理介绍

循环神经网络（Recurrent Neural Network, RNN）是一类以序列（sequence）数据为输入，在序列的演进方向进行递归（recursion）且所有节点（循环单元）按链式连接的神经网络。下图为RNN的一般结构：

![RNN-0](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/tutorials/application/source_zh_cn/nlp/images/0-RNN-0.png)

图示左侧为一个RNN Cell循环，右侧为RNN的链式连接平铺。实际上不管是单个RNN Cell还是一个RNN网络，都只有一个Cell的参数，在不断进行循环计算中更新。

由于RNN的循环特性，和自然语言文本的序列特性(句子是由单词组成的序列)十分匹配，因此被大量应用于自然语言处理研究中。下图为RNN的结构拆解：

![RNN](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/tutorials/application/source_zh_cn/nlp/images/0-RNN.png)

RNN单个Cell的结构简单，因此也造成了梯度消失(Gradient Vanishing)问题，具体表现为RNN网络在序列较长时，在序列尾部已经基本丢失了序列首部的信息。为了克服这一问题，LSTM(Long short term memory)被提出，通过门控机制(Gating Mechanism)来控制信息流在每个循环步中的留存和丢弃。下图为LSTM的结构拆解：

![LSTM](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/tutorials/application/source_zh_cn/nlp/images/0-LSTM.png)

本节我们选择LSTM变种而不是经典的RNN做特征提取，来规避梯度消失问题，并获得更好的模型效果。下面来看MindSpore中nn.LSTM对应的公式：

$$h_{0:t}, (h_t, c_t) = \text{LSTM}(x_{0:t}, (h_0, c_0))$$

这里nn.LSTM隐藏了整个循环神经网络在序列时间步(Time step)上的循环，送入输入序列、初始状态，即可获得每个时间步的隐状态(hidden state)拼接而成的矩阵，以及最后一个时间步对应的隐状态。我们使用最后的一个时间步的隐状态作为输入句子的编码特征，送入下一层。

在循环神经网络计算的每一次循环，成为一个Time step。在送入文本序列时，一个Time step对应一个单词。因此在本例中，LSTM的输出$h_{0:t}$对应每个单词的隐状态集合，$h_t$和$c_t$对应最后一个单词对应的隐状态。

## 3 实验环境
### 实验环境要求
在动手进行实践之前，需要注意以下几点：
* 确保实验环境正确安装，包括安装MindSpore。安装过程：首先登录[MindSpore官网安装页面](https://www.mindspore.cn/install)，根据安装指南下载安装包及查询相关文档。同时，官网环境安装也可以按下表说明找到对应环境搭建文档链接，根据环境搭建手册配置对应的实验环境。
* 推荐使用交互式的计算环境Jupyter Notebook，其交互性强，易于可视化，适合频繁修改的数据分析实验环境。
* 实验也可以在华为云一站式的AI开发平台ModelArts上完成。
* 推荐实验环境：MindSpore版本=MindSpore 2.4；Python环境=3.11。


|  硬件平台 |  操作系统  |           软件环境           | 开发环境 | 环境搭建链接 |
| :-----:| :----: |:------------------------:|:----:   |:----:   |
| CPU | Windows-x64 | MindSpore2.4 Python3.11  | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.1节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| Ascend 910  | Linux-x86_64| MindSpore2.4 Python3.11  | JupyterNotebook |[MindSpore环境搭建实验手册第四章](./MindSpore环境搭建实验手册.docx)|
## 4 数据处理
本文使用了2930首五言律诗作为训练数据集，每一行数据为一首诗，这里使用的诗的作者和诗名都已经被删去。

数据集保存在文件 `poetry.txt` 中，其中的部分古诗如下：

```text
山山浮翠远，处处落红深。独立柴门外，长歌托素心。
翠华重幸日，瑞气霭龙牙。元老频承宠，宸章特赐嘉。
调和归静穆，暑雨绝咨嗟。共仰明良会，唐虞岂有加。
一字同华衮，天香遍齿牙。光辉悬日月，矍铄入褒嘉。
...
```

数据集下载:

使用 `download` 接口下载数据集，并将下载后的数据集自动解压到当前目录下。数据下载之前需要使用 `pip install download` 安装 `download` 包。

In [1]:
from download import download
url = "https://ascend-professional-construction-dataset.obs.cn-north-4.myhuaweicloud.com:443/NLP/LSTM_generate_poem_data.zip"
download(url, "./", kind="zip", replace=True)


file_sizes: 100%|██████████████████████████| 3.20M/3.20M [00:01<00:00, 1.73MB/s]
Extracting zip file...
Successfully downloaded / unzipped to ./


'./'

## 5 模型构建
（1）导入Python库&模块并配置运行信息

导入MindSpore模块和辅助模块。

代码如下：

In [1]:
#导入mindspore模块和辅助模块
from mindspore import nn, ops, Tensor, save_checkpoint

(2)定义参数变量

In [2]:
#设置参数变量
hidden_dim = 128
n_layers = 2
batch_size = 64
lr = 0.003
total_epoch = 5

(3)数据的读取与处理

先按行读取数据集，再使用 `rstrip` 方法去除后缀字符，然后过滤掉含有特殊字符的文本，然后用 `word2vec` 将文本词向量化。

Word2Vec 是一种将词转为向量的方法，是在2013年的论文 [Efficient Estimation of Word Representations inVector Space](https://arxiv.org/pdf/1301.3781.pdf) 中提出的，作者来自google。

比较早出现的将词转为向量的方法是独热编码（One-Hot）。使用独热编码构建独热向量很容易，但通常不是一个好的选择。一个主要原因是独热向量不能准确表达不同词之间的相似度，而且在大量数据的情况下会出现数据的维度灾难。

Word2Vec 方法刚好克服了独热编码的缺陷。它将每个词映射到一个固定长度（长度为可设置的超参数）的向量，这些向量能很好地表达不同词之间的相似性和类比关系。该方法包含两种模型算法：skip-gram 和 CBOW，它们的最大区别是 skip-gram 是通过中心词去预测中心词周围的词，而 CBOW 是通过周围的词去预测中心词。

此方法需要通过 `gensim.models.word2vec` 来使用，上述的两种训练模型可以通过超参数 `sg` 来设置，默认为 CBOW。这里介绍 Work2Vec 的原因是它可以替换下文的模型的 Embedding 层。

`gensim` 库需手动安装，命令如下：`pip install gensim`

具体情况见如下代码：

In [3]:
import os
import pickle
import mindspore as ms
from gensim.models.word2vec import Word2Vec
dataset_path = './LSTM_generate_poem_data/poetry_5.txt'
vec_params_file = 'vec_params_5.pkl'  # 保存词向量训练后的权重文件名字
ms.set_context(device_target='CPU', mode=ms.PYNATIVE_MODE, device_id=0)
def data_pre(dataset_path, vec_params_file):
    # 特殊字符列表
    forbidden_words = ['（', '）', '(', ')', '__', '《', '》', '【', '】', '[', ']']
    poetry_src = []
    with open(dataset_path, encoding='utf-8') as f:
        lines = f.readlines()
    for poem in lines:
        forbidden_poem = [word in poem for word in forbidden_words]
        # 过滤掉含有特殊字符的诗句
        if sum(forbidden_poem) > 0:
            continue
        # 去除行尾换行符
        poem = poem.rstrip()
        poetry_src.append(poem)
    if os.path.exists(vec_params_file):
        return poetry_src, pickle.load(open(vec_params_file, "rb"))
    model = Word2Vec(poetry_src, vector_size=100, min_count=1, workers=6)
    pickle.dump((model.syn1neg, model.wv.key_to_index, model.wv.index_to_key), open(vec_params_file, "wb"))
    return poetry_src, (model.syn1neg, model.wv.key_to_index, model.wv.index_to_key)
all_data, (word_vec, word_2_index, index_2_word) = data_pre(dataset_path, vec_params_file)
vocab_size, embedding_dim = word_vec.shape
print('vocab_size: ', vocab_size)

vocab_size:  4436


完成预处理操作后，需将其加入到数据集处理流水线中。类方法 `PoetryDataGenerator` 将数据拆分为模型的输入数据和用于计算loss的标签，并转换为 Tensor 格式，`__getitem__` 方法将数据变为可迭代对象。函数 `create_poetry_dataset` 是将可迭代的数据使用 `GeneratorDataset` 封装成MindSpore 的 dataset 对象，然后可以使用dataset 对象的 `get_batch_size` 属性获得数据集的大小 ，使用 `batch` 批操作将数据批量化。

In [4]:
import mindspore.dataset as ds
from mindspore import Tensor
class PoetryDataGenerator(object):
    def __init__(self, all_data, word_vec, word_2_index):
        self.all_data = all_data
        self.word_vec = word_vec
        self.word_2_index = word_2_index
    # 获取一条数据, 并作处理
    def __getitem__(self, index):
        a_poetry_words = self.all_data[index]
        a_poetry_index = [self.word_2_index[word] for word in a_poetry_words]
        xs_index = a_poetry_index[:-1]
        ys_index = a_poetry_index[1:]
        xs_embedding = self.word_vec[xs_index]

        return Tensor(xs_embedding), Tensor(ys_index, ms.int32)
    def __len__(self):
        return len(self.all_data)
def create_poetry_dataset(batch_size, all_data, word_vec, word_2_index):
    dt = PoetryDataGenerator(all_data, word_vec, word_2_index)
    de = ds.GeneratorDataset(dt, ["inputs", "label"], shuffle=True)
    size = de.get_dataset_size()
    de = de.batch(batch_size, drop_remainder=True)
    return de, size
dataset, size = create_poetry_dataset(batch_size, all_data, word_vec, word_2_index)
total_step = dataset.get_dataset_size()
data1 = next(dataset.create_dict_iterator())
data1_shape = data1["inputs"].shape
print('Input data size:', data1_shape)
print('Size:', size)
print('Batchsize:', batch_size)
print('Total_step:', total_step)

Input data size: (64, 23, 100)
Size: 2930
Batchsize: 64
Total_step: 45


(4)模型构建

完成数据集的处理后，我们设计用于文本生成的模型结构。一般情况下首先需要使用`nn.Embedding`层加载词向量；然后使用RNN循环神经网络做特征提取；最后将RNN连接至一个全连接层，即 `nn.Dense` ，将特征转化为需要预测的词表大小的结果，用于后续进行模型优化训练。整体模型结构如下：

```text
nn.Embedding -> nn.RNN -> nn.Dense
```

这里使用能够一定程度规避RNN梯度消失问题的变种 LSTM(Long short term memory) 做特征提取层。下面对模型进行详解：

Embedding 层又可称为 EmbeddingLookup 层，其作用是使用 index id 对权重矩阵对应id的向量进行查找，当输入为一个由 index id 组成的序列时，则查找并返回一个相同长度的矩阵，例如：

```text
embedding = nn.Embedding(1000, 100) # 词表大小(index的取值范围)为1000，表示向量的size为100
input shape: (1, 16)                # 序列长度为16
output shape：(1, 16, 100)
```

这里使用 Work2Vec 方法处理的数据来替代 Embedding 层，可以达到相同的效果。

在经过LSTM编码获取句子特征后，将其送入一个全连接层，即`nn.Dense`，将特征维度变换为需要预测的词表大小，然后使用 `ops.argmax` 选取最大的结果的位置，再对应到词表找到相应的词汇即为模型预测结果。

In [5]:
from mindspore import nn, ops, Tensor
class LSTMPoetryModel(nn.Cell):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, n_layers, batch_size):
        super().__init__()
        self.vocab_size = vocab_size  # 词汇表大小
        self.embedding_dim = embedding_dim  # 嵌入维度
        self.hidden_dim = hidden_dim  # LSTM隐藏层维度
        self.n_layers = n_layers  # LSTM层数
        self.batch_size = batch_size  # 批次大小
# 定义LSTM层
        self.rnn = nn.LSTM(self.embedding_dim,
                           self.hidden_dim,
                           num_layers=self.n_layers,
                           batch_first=True)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Dense(self.hidden_dim, self.vocab_size)
    def construct(self, inputs, h_0=None, c_0=None):
# 如果没有提供初始的隐藏状态和细胞状态，就使用零张量进行初始化
        if h_0 == None or c_0 == None:
            h_0 = Tensor(np.zeros((self.n_layers, self.batch_size, self.hidden_dim)), ms.float32)
            c_0 = Tensor(np.zeros((self.n_layers, self.batch_size, self.hidden_dim)), ms.float32)
# 前向计算
        outputs, (h_0, c_0) = self.rnn(inputs, (h_0, c_0))
        outputs = self.dropout(outputs)
        outputs = ops.reshape(outputs, (outputs.shape[0] * outputs.shape[1], outputs.shape[2]))
        output = self.fc(outputs)
        return output, h_0, c_0
# 模型实例化
rnn_net = LSTMPoetryModel(vocab_size, embedding_dim, hidden_dim, n_layers, batch_size)
#选择交叉熵损失函数 nn.CrossEntropyLoss()，优化器选择 nn.Adam
loss_fn = nn.CrossEntropyLoss()
opt = nn.Adam(rnn_net.trainable_params(), learning_rate=lr)

[WARNING] ME(257726:140128658933568,MainProcess):2025-01-13-10:15:43.477.398 [mindspore/nn/layer/basic.py:177] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.


## 6 模型训练


训练过程主要是先前向计算损失，然后根据损失值计算梯度，再根据梯度更新模型参数，最后打印损失保存权重文件。

In [6]:
import time
import numpy as np
from mindspore import save_checkpoint

#定义前向传播函数
def forward_fn(datas):
    output, h_0, c_0 = rnn_net(datas["inputs"])
    loss = loss_fn(output, datas["label"].reshape(-1))
    return loss
grad_fn = ms.value_and_grad(forward_fn, None, rnn_net.trainable_params())
def train_step(data1):
    loss, grads = grad_fn(data1)
    opt(grads)
    return loss
#训练过程
def train():
    print('Start training!')
    epochs_loss = []
    for epoch in range(total_epoch):
        start = time.time()
        steps_loss = []
# 遍历数据集
        for step, data in enumerate(dataset.create_dict_iterator()):
            start1 = time.time()
            loss = train_step(data)
            steps_loss.append(loss.asnumpy())
            end1 = time.time()
            if step % 10 == 0:
# 打印训练信息
                print(f"Epoch:[{int(epoch+1):>3d}/{int(total_epoch):>3d}], "
                      f"step:[{int(step):>4d}/{int(total_step):>4d}], "
                      f"loss:{loss.asnumpy():>4f} , "
                      f"time:{(end1 - start1):>3f}s, "
                      f"lr:{lr:>6f}")
        epochs_loss.append(np.mean(steps_loss))
        end = time.time()
        print(f"Epoch:[{int(epoch + 1):>3d}/{int(total_epoch):>3d}], "
              f"loss:{epochs_loss[epoch]:>4f} , "
              f"time:{(end - start):>3f}s, ")
        if (epoch+1) % 10 == 0:
            save_checkpoint(rnn_net, f"./LSTM_poetry_{epoch+1}.ckpt")
    print('Training completed!')
train()

Start training!
Epoch:[  1/  5], step:[   0/  45], loss:8.400741 , time:1.094344s, lr:0.003000
Epoch:[  1/  5], step:[  10/  45], loss:6.941288 , time:0.394345s, lr:0.003000
Epoch:[  1/  5], step:[  20/  45], loss:6.929317 , time:0.451954s, lr:0.003000
Epoch:[  1/  5], step:[  30/  45], loss:6.897366 , time:0.346353s, lr:0.003000
Epoch:[  1/  5], step:[  40/  45], loss:6.861903 , time:0.397491s, lr:0.003000
Epoch:[  1/  5], loss:7.108612 , time:19.920826s, 
Epoch:[  2/  5], step:[   0/  45], loss:6.737153 , time:0.388602s, lr:0.003000
Epoch:[  2/  5], step:[  10/  45], loss:6.778171 , time:0.384442s, lr:0.003000
Epoch:[  2/  5], step:[  20/  45], loss:6.709825 , time:0.418344s, lr:0.003000
Epoch:[  2/  5], step:[  30/  45], loss:6.773804 , time:0.404978s, lr:0.003000
Epoch:[  2/  5], step:[  40/  45], loss:6.697762 , time:0.368519s, lr:0.003000
Epoch:[  2/  5], loss:6.767356 , time:18.497317s, 
Epoch:[  3/  5], step:[   0/  45], loss:6.726222 , time:0.382242s, lr:0.003000
Epoch:[  3/  

## 7 模型测试

最后我们设计一个预测函数，实现随机初始化一首诗的第一个字然后生成一首五言诗。具体代码如下:

In [9]:
from mindspore import load_checkpoint, load_param_into_net
def eval():
    result = ""
# 随机选择一个词的索引作为起始词
    word_index = np.random.randint(0, vocab_size, 1)[0]
    result += index_2_word[word_index]
    print('Start word:', result)
# 初始化LSTM的隐藏状态和细胞状态
    h_0 = Tensor(np.zeros((n_layers, 1, hidden_dim)), ms.float32)
    c_0 = Tensor(np.zeros((n_layers, 1, hidden_dim)), ms.float32)
# 遍历输入数据的每个时间步
    for i in range(data1_shape[1]):
        inputs = Tensor([[word_vec[word_index]]])
        output, h_0, c_0 = rnn_net(inputs, h_0, c_0)
        word_index = int(ops.argmax(output))
        result += index_2_word[word_index]
    print('Predict result:', result)
# 加载模型参数
param_dict = load_checkpoint('LSTM_generate_poem_data/LSTM_poetry.ckpt')
# 将参数加载到网络中
load_param_into_net(rnn_net, param_dict)
# 调用评估函数
eval()

Start word: 桓
Predict result: 桓载志霭叛。穷士柳重人，屈船蘸席图。番甘度共角，


## 8 实验总结

以上便完成了MindSpore生成文本的全过程。通过本次体验，可以全面了解如何使用MindSpore进行LSTM算法构建，并可以熟悉循环神经网络的结构，文本生成的过程以及网络模型的训练和预测，掌握 MindSpore 的相关操作等。